In [ ]:
import networkx as nx
from networkx.algorithms.bipartite import sets
from numpy import *
from networkx.algorithms import bipartite
import itertools
import random
import matplotlib.pyplot as plt

In [ ]:
# Граф Эрдьёша-Реньи

def random_graph(n:int, p:double, rnd=None) -> nx.Graph:
  gr = nx.Graph()
  N_range = range(n)
  gr.add_nodes_from(N_range)
   
  for pair in itertools.permutations(N_range, 2):
    if random.random()  < p:
      gr.add_edge(*pair)
   
  return gr

In [ ]:
#Генерация сочетаний из n по k без повторений

def comb(n, k):   
    a = list(range(0, k))
    yield a

    while True:
        i = k - 1
        while i >= 0 and a[i] > n - k + i - 1:
            i -= 1
        if i < 0:
            return

        a[i] += 1
        for j in range(i + 1, k):
            a[j] = a[j - 1] + 1

        yield a

In [ ]:
def is_bipartite_node_set(G, nodes):
    S = set(nodes)
    for CC in (G.subgraph(c).copy() for c in nx.connected_components(G)):
        X, Y = sets(CC)
        if not ((X.issubset(S) and Y.isdisjoint(S)) or
                (Y.issubset(S) and X.isdisjoint(S))):
            return False
    return True

In [ ]:
def color(G):
    if G.is_directed():
        import itertools

        def neighbors(v):
            return itertools.chain.from_iterable([G.predecessors(v),
                                                  G.successors(v)])
    else:
        neighbors = G.neighbors

    color = {}
    for n in G:  # handle disconnected graphs
        if n in color or len(G[n]) == 0:  # skip isolates
            continue
        queue = [n]
        color[n] = 1  # nodes seen with color (1 or 0)
        while queue:
            v = queue.pop()
            c = 1 - color[v]  # opposite color of node v
            for w in neighbors(v):
                if w in color:
                    if color[w] == color[v]:
                        raise nx.NetworkXError("Graph is not bipartite.")
                else:
                    color[w] = c
                    queue.append(w)
    # color isolates with 0
    color.update(dict.fromkeys(nx.isolates(G), 0))
    return color

In [ ]:
def is_bipartite(G):
    try:
        color(G)
        return True
    except nx.NetworkXError:
        return False


In [ ]:
def min_weighted_vertex_cover(G, weight=None):
    cost = dict(G.nodes(data=weight, default=1))
    # While there are uncovered edges, choose an uncovered and update
    # the cost of the remaining edges.
    for u, v in G.edges():
        min_cost = min(cost[u], cost[v])
        cost[u] -= min_cost
        cost[v] -= min_cost
    return {u for u, c in cost.items() if c == 0}